<a href="https://colab.research.google.com/github/fuse999/DS-Unit-2-Regression-Classification/blob/master/module3/Riley_assignment_regression_classification_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Regression & Classification, Module 3

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

Instead, predict property sales prices for **One Family Dwellings** (`BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'`). 

Use a subset of the data where the **sale price was more than \\$100 thousand and less than $2 million.** 

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.

- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Fit a ridge regression model with multiple features.
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.


## Stretch Goals
- [ ] Add your own stretch goal(s) !
- [ ] Instead of `RidgeRegression`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `RidgeRegression`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
import os, sys
in_colab = 'google.colab' in sys.modules

# If you're in Colab...
if in_colab:
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification.git
    !git pull origin master
    
    # Install required python packages
    !pip install -r requirements.txt
    
    # Change into directory for module
    os.chdir('module3')

In [0]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv('../data/condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [0]:
# Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.

In [0]:
import numpy as np
from sklearn.feature_selection import f_regression, SelectKBest

In [0]:
# drop Sale Price Values Under 100,000

df = df[(df['SALE_PRICE'] > 100000) & (df['SALE_PRICE'] < 2000000)]

In [0]:
df = df.drop(columns=['EASE-MENT', 'APARTMENT_NUMBER'])

In [205]:
df_copy.isnull().sum()

BOROUGH                            0
NEIGHBORHOOD                       0
BUILDING_CLASS_CATEGORY            0
TAX_CLASS_AT_PRESENT               0
BLOCK                              0
LOT                                0
BUILDING_CLASS_AT_PRESENT          0
ADDRESS                            0
ZIP_CODE                           0
RESIDENTIAL_UNITS                  0
COMMERCIAL_UNITS                   0
TOTAL_UNITS                        0
LAND_SQUARE_FEET                  37
GROSS_SQUARE_FEET                  0
YEAR_BUILT                        22
TAX_CLASS_AT_TIME_OF_SALE          0
BUILDING_CLASS_AT_TIME_OF_SALE     0
SALE_PRICE                         0
SALE_DATE                          0
dtype: int64

In [204]:
from datetime import datetime
df['month'] = pd.DatetimeIndex(df['SALE_DATE']).month

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ADDRESS,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE,month
44,3,OCEAN PARKWAY-NORTH,01 ONE FAMILY DWELLINGS,1,5495,801,A9,4832 BAY PARKWAY,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019,1
61,4,QUEENS VILLAGE,01 ONE FAMILY DWELLINGS,1,7918,72,A1,80-23 232ND STREET,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,01/01/2019,1
66,1,MIDTOWN EAST,10 COOPS - ELEVATOR APARTMENTS,2,1347,18,D4,"345 EAST 54TH ST, 3B",10022.0,0.0,0.0,0.0,0,0.0,1960.0,2,D4,330000,01/02/2019,1
67,1,UPPER EAST SIDE (79-96),10 COOPS - ELEVATOR APARTMENTS,2,1491,62,D4,"16 EAST 80TH STREET, 2A",10075.0,0.0,0.0,0.0,0,0.0,1925.0,2,D4,600000,01/02/2019,1
71,1,UPPER WEST SIDE (59-79),13 CONDOS - ELEVATOR APARTMENTS,2,1171,2200,R4,"240 RIVERSIDE BOULEVARD, 4 F",10069.0,1.0,0.0,1.0,0,827.0,2004.0,2,R4,1250000,01/02/2019,1


In [0]:
df = df.dropna()

In [207]:
df.isnull().sum()

BOROUGH                           0
NEIGHBORHOOD                      0
BUILDING_CLASS_CATEGORY           0
TAX_CLASS_AT_PRESENT              0
BLOCK                             0
LOT                               0
BUILDING_CLASS_AT_PRESENT         0
ADDRESS                           0
ZIP_CODE                          0
RESIDENTIAL_UNITS                 0
COMMERCIAL_UNITS                  0
TOTAL_UNITS                       0
LAND_SQUARE_FEET                  0
GROSS_SQUARE_FEET                 0
YEAR_BUILT                        0
TAX_CLASS_AT_TIME_OF_SALE         0
BUILDING_CLASS_AT_TIME_OF_SALE    0
SALE_PRICE                        0
SALE_DATE                         0
month                             0
dtype: int64

In [0]:
df['LAND_SQUARE_FEET'] = df['LAND_SQUARE_FEET'].str.replace(',', '').astype(int)

In [0]:
df['created'] = pd.to_datetime(df['SALE_DATE'], infer_datetime_format=True)

cutoff = pd.to_datetime('2019-03-01')
train = df[df.created < cutoff]
test = df[df['month'] == 4]

In [0]:
df_data = df

In [211]:
test.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ADDRESS,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE,month,created
18167,1,CHELSEA,13 CONDOS - ELEVATOR APARTMENTS,2,695,1562,R4,"555 WEST 23RD STREET, N11E",10011.0,1.0,0.0,1.0,0,670.0,2005.0,2,R4,949000,04/01/2019,4,2019-04-01
18170,1,CHELSEA,17 CONDO COOPS,2,721,1008,R9,"465 WEST 23RD STREET, RCU1",10011.0,0.0,0.0,0.0,0,0.0,1932.0,2,R9,1100000,04/01/2019,4,2019-04-01
18173,1,FINANCIAL,13 CONDOS - ELEVATOR APARTMENTS,2,31,1050,R4,"75 WALL STREET, 21I",10005.0,1.0,0.0,1.0,0,438.0,1987.0,2,R4,675000,04/01/2019,4,2019-04-01
18174,1,GRAMERCY,10 COOPS - ELEVATOR APARTMENTS,2,880,42,D4,"330 THIRD AVENUE, 18K",10010.0,0.0,0.0,0.0,0,0.0,1964.0,2,D4,485000,04/01/2019,4,2019-04-01
18175,1,GRAMERCY,13 CONDOS - ELEVATOR APARTMENTS,2,878,1010,R4,"121 EAST 22ND STREET, N204",10010.0,1.0,0.0,1.0,22202,773.0,1952.0,2,R4,1486645,04/01/2019,4,2019-04-01


In [212]:
df.describe()

,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,SALE_PRICE,month
count,13352.000000,13352.000000,13352.000000,13352.000000,13352.000000,13352.00000,13352.000000,13352.000000,13352.000000,13352.000000,1.335200e+04,13352.000000
mean,4660.654733,343.275689,10821.380467,1.155707,0.143574,1.52846,4737.368484,1692.525315,1835.692256,1.579838,7.160286e+05,2.428850
std,3734.958881,609.322161,1030.166073,3.347782,1.475381,5.73121,21981.406174,7579.854817,461.483713,0.697368,4.065290e+05,1.113323
min,3.000000,1.000000,0.000000,0.000000,-1.000000,0.00000,0.000000,0.000000,0.000000,1.000000,1.000440e+05,1.000000
25%,1458.000000,20.000000,10310.000000,0.000000,0.000000,0.00000,0.000000,0.000000,1923.000000,1.000000,4.140000e+05,1.000000
50%,3900.000000,49.000000,11213.000000,1.000000,0.000000,1.00000,1919.000000,1127.000000,1945.000000,1.000000,6.320000e+05,2.000000
75%,6837.000000,316.000000,11365.000000,2.000000,0.000000,2.00000,3000.000000,1870.250000,1965.000000,2.000000,9.150000e+05,3.000000
max,16350.000000,9002.000000,11697.000000,155.000000,60.000000,156.00000,484555.000000,322160.000000,2018.000000,4.000000,1.999877e+06,4.000000


In [0]:
# Do one-hot encoding of categorical features.

In [213]:
train.select_dtypes(include='number').describe().T

,count,mean,std,min,25%,50%,75%,max
BLOCK,6867.0,4615.587156,3695.206680,3.0,1450.0,3872.0,6783.0,16350.0
LOT,6867.0,325.593127,589.122533,1.0,20.0,48.0,207.5,9002.0
ZIP_CODE,6867.0,10810.208970,1043.455339,0.0,10309.0,11210.0,11364.0,11697.0
RESIDENTIAL_UNITS,6867.0,1.128295,2.847108,0.0,0.0,1.0,2.0,155.0
COMMERCIAL_UNITS,6867.0,0.090578,0.900663,0.0,0.0,0.0,0.0,50.0
TOTAL_UNITS,6867.0,1.305519,3.736886,0.0,0.0,1.0,2.0,156.0
LAND_SQUARE_FEET,6867.0,4999.647881,23111.454328,0.0,0.0,1950.0,3000.0,484555.0
GROSS_SQUARE_FEET,6867.0,1452.797583,5824.024182,0.0,0.0,1144.0,1836.0,244619.0
YEAR_BUILT,6867.0,1841.360419,449.924376,0.0,1923.0,1945.0,1965.0,2018.0
TAX_CLASS_AT_TIME_OF_SALE,6867.0,1.560216,0.686291,1.0,1.0,1.0,2.0,4.0


In [151]:
train.select_dtypes(exclude='number').describe().T.sort_values(by='unique')

,count,unique,top,freq,first,last
BOROUGH,12062,5,4,3886,NaN,NaN
TAX_CLASS_AT_PRESENT,12062,10,1,5954,NaN,NaN
NEIGHBORHOOD,12062,11,OTHER,9952,NaN,NaN
BUILDING_CLASS_CATEGORY,12062,41,01 ONE FAMILY DWELLINGS,2768,NaN,NaN
SALE_DATE,12062,59,01/24/2019,480,NaN,NaN
created,12062,59,2019-01-24 00:00:00,480,2019-01-01 00:00:00,2019-02-28 00:00:00
BUILDING_CLASS_AT_TIME_OF_SALE,12062,128,D4,1745,NaN,NaN
BUILDING_CLASS_AT_PRESENT,12062,129,D4,1745,NaN,NaN
APARTMENT_NUMBER,2559,1118,2A,46,NaN,NaN
LAND_SQUARE_FEET,12055,2554,0,3827,NaN,NaN


In [214]:
train.groupby('BUILDING_CLASS_CATEGORY')['SALE_PRICE'].mean()

BUILDING_CLASS_CATEGORY
01 ONE FAMILY DWELLINGS                       6.234864e+05
02 TWO FAMILY DWELLINGS                       7.714372e+05
03 THREE FAMILY DWELLINGS                     9.431418e+05
04 TAX CLASS 1 CONDOS                         5.418484e+05
05 TAX CLASS 1 VACANT LAND                    7.628139e+05
06 TAX CLASS 1 - OTHER                        7.020833e+05
07 RENTALS - WALKUP APARTMENTS                1.062822e+06
08 RENTALS - ELEVATOR APARTMENTS              1.064848e+06
09 COOPS - WALKUP APARTMENTS                  4.723739e+05
10 COOPS - ELEVATOR APARTMENTS                5.568812e+05
11A CONDO-RENTALS                             1.140000e+06
12 CONDOS - WALKUP APARTMENTS                 6.599400e+05
13 CONDOS - ELEVATOR APARTMENTS               8.781459e+05
14 RENTALS - 4-10 UNIT                        1.108786e+06
15 CONDOS - 2-10 UNIT RESIDENTIAL             9.201003e+05
16 CONDOS - 2-10 UNIT WITH COMMERCIAL UNIT    9.201647e+05
17 CONDO COOPS                  

In [0]:
target = 'SALE_PRICE'
high_cardinality = ['LAND_SQUARE_FEET', 'NEIGHBORHOOD', 'BUILDING_CLASS_AT_PRESENT', 'SALE_DATE', 'ADDRESS', 'created']
features = train.columns.drop([target] + high_cardinality)

X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

In [172]:
X_train.head()

,BOROUGH,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,month
0,1,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,10011.0,1.0,0.0,1.0,1979.0,2007.0,2,R4,1
1,1,21 OFFICE BUILDINGS,4,812,68,NaN,10018.0,0.0,6.0,6.0,15435.0,1920.0,4,O5,1
2,1,21 OFFICE BUILDINGS,4,839,69,NaN,10018.0,0.0,7.0,7.0,11332.0,1930.0,4,O5,1
3,1,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,10014.0,1.0,0.0,1.0,500.0,0.0,2,R4,1
4,1,15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,10065.0,1.0,0.0,1.0,6406.0,0.0,2,R1,1


In [0]:
import category_encoders as ce
encodr = ce.OneHotEncoder(use_cat_names=True)
X_train = encodr.fit_transform(X_train)
X_test = encodr.transform(X_test)

In [219]:
print(X_train.shape)
X_train.head()

(6867, 147)


,BOROUGH_3,BOROUGH_4,BOROUGH_1,BOROUGH_2,BOROUGH_5,BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS,BUILDING_CLASS_CATEGORY_10 COOPS - ELEVATOR APARTMENTS,BUILDING_CLASS_CATEGORY_13 CONDOS - ELEVATOR APARTMENTS,BUILDING_CLASS_CATEGORY_02 TWO FAMILY DWELLINGS,BUILDING_CLASS_CATEGORY_03 THREE FAMILY DWELLINGS,BUILDING_CLASS_CATEGORY_41 TAX CLASS 4 - OTHER,BUILDING_CLASS_CATEGORY_08 RENTALS - ELEVATOR APARTMENTS,BUILDING_CLASS_CATEGORY_44 CONDO PARKING,BUILDING_CLASS_CATEGORY_07 RENTALS - WALKUP APARTMENTS,BUILDING_CLASS_CATEGORY_47 CONDO NON-BUSINESS STORAGE,BUILDING_CLASS_CATEGORY_05 TAX CLASS 1 VACANT LAND,BUILDING_CLASS_CATEGORY_43 CONDO OFFICE BUILDINGS,BUILDING_CLASS_CATEGORY_09 COOPS - WALKUP APARTMENTS,BUILDING_CLASS_CATEGORY_04 TAX CLASS 1 CONDOS,BUILDING_CLASS_CATEGORY_17 CONDO COOPS,BUILDING_CLASS_CATEGORY_29 COMMERCIAL GARAGES,BUILDING_CLASS_CATEGORY_15 CONDOS - 2-10 UNIT RESIDENTIAL,BUILDING_CLASS_CATEGORY_12 CONDOS - WALKUP APARTMENTS,BUILDING_CLASS_CATEGORY_27 FACTORIES,BUILDING_CLASS_CATEGORY_22 STORE BUILDINGS,BUILDING_CLASS_CATEGORY_31 COMMERCIAL VACANT LAND,BUILDING_CLASS_CATEGORY_45 CONDO HOTELS,BUILDING_CLASS_CATEGORY_32 HOSPITAL AND HEALTH FACILITIES,BUILDING_CLASS_CATEGORY_14 RENTALS - 4-10 UNIT,BUILDING_CLASS_CATEGORY_06 TAX CLASS 1 - OTHER,BUILDING_CLASS_CATEGORY_46 CONDO STORE BUILDINGS,BUILDING_CLASS_CATEGORY_21 OFFICE BUILDINGS,BUILDING_CLASS_CATEGORY_37 RELIGIOUS FACILITIES,BUILDING_CLASS_CATEGORY_33 EDUCATIONAL FACILITIES,BUILDING_CLASS_CATEGORY_30 WAREHOUSES,BUILDING_CLASS_CATEGORY_11A CONDO-RENTALS,BUILDING_CLASS_CATEGORY_36 OUTDOOR RECREATIONAL FACILITIES,BUILDING_CLASS_CATEGORY_34 THEATRES,BUILDING_CLASS_CATEGORY_16 CONDOS - 2-10 UNIT WITH COMMERCIAL UNIT,BUILDING_CLASS_CATEGORY_26 OTHER HOTELS,...,BUILDING_CLASS_AT_TIME_OF_SALE_S3,BUILDING_CLASS_AT_TIME_OF_SALE_C5,BUILDING_CLASS_AT_TIME_OF_SALE_G0,BUILDING_CLASS_AT_TIME_OF_SALE_S4,BUILDING_CLASS_AT_TIME_OF_SALE_RK,BUILDING_CLASS_AT_TIME_OF_SALE_O7,BUILDING_CLASS_AT_TIME_OF_SALE_M1,BUILDING_CLASS_AT_TIME_OF_SALE_K4,BUILDING_CLASS_AT_TIME_OF_SALE_S9,BUILDING_CLASS_AT_TIME_OF_SALE_GW,BUILDING_CLASS_AT_TIME_OF_SALE_Z0,BUILDING_CLASS_AT_TIME_OF_SALE_C9,BUILDING_CLASS_AT_TIME_OF_SALE_F1,BUILDING_CLASS_AT_TIME_OF_SALE_W9,BUILDING_CLASS_AT_TIME_OF_SALE_G1,BUILDING_CLASS_AT_TIME_OF_SALE_S5,BUILDING_CLASS_AT_TIME_OF_SALE_O2,BUILDING_CLASS_AT_TIME_OF_SALE_E1,BUILDING_CLASS_AT_TIME_OF_SALE_C7,BUILDING_CLASS_AT_TIME_OF_SALE_O5,BUILDING_CLASS_AT_TIME_OF_SALE_M9,BUILDING_CLASS_AT_TIME_OF_SALE_RR,BUILDING_CLASS_AT_TIME_OF_SALE_Q9,BUILDING_CLASS_AT_TIME_OF_SALE_G3,BUILDING_CLASS_AT_TIME_OF_SALE_W2,BUILDING_CLASS_AT_TIME_OF_SALE_F4,BUILDING_CLASS_AT_TIME_OF_SALE_O8,BUILDING_CLASS_AT_TIME_OF_SALE_J9,BUILDING_CLASS_AT_TIME_OF_SALE_R7,BUILDING_CLASS_AT_TIME_OF_SALE_W8,BUILDING_CLASS_AT_TIME_OF_SALE_S0,BUILDING_CLASS_AT_TIME_OF_SALE_F9,BUILDING_CLASS_AT_TIME_OF_SALE_R8,BUILDING_CLASS_AT_TIME_OF_SALE_D9,BUILDING_CLASS_AT_TIME_OF_SALE_O1,BUILDING_CLASS_AT_TIME_OF_SALE_H7,BUILDING_CLASS_AT_TIME_OF_SALE_R5,BUILDING_CLASS_AT_TIME_OF_SALE_O9,BUILDING_CLASS_AT_TIME_OF_SALE_P2,month
44,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
61,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
66,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
67,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
71,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [0]:
# Do feature selection with SelectKBest.

In [220]:
selector = SelectKBest(score_func=f_regression, k=12)

X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)
X_train_selected.shape, X_test_selected.shape

((6867, 12), (2940, 12))

In [0]:
# Which features were selected?
all_names = X_train.columns
selected_mask = selector.get_support()
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
    print(name)
    
print('\n')
print('Features not selected:')
for name in unselected_names:
    print(name)

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

for k in range(1, len(X_train.columns)+1):
    
    print(f'{k} features')
    
    selector = SelectKBest(score_func=f_regression, k=k)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)
    
    model = LinearRegression()
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)
    
    mae = mean_absolute_error(y_test, y_pred)
    print(f'Test MAE: ${mae:,.0f} \n')

In [0]:
# 60, 61, 62

In [0]:
# Fit a ridge regression model with multiple features.

In [226]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
ridge_reg = Ridge().fit(X_train_selected, y_train)
mean_squared_error(y_train, ridge_reg.predict(X_train_selected))

94954661296.06822

In [227]:
ridge_reg_split = Ridge().fit(X_train_selected, y_train)
mean_squared_error(y_test, ridge_reg_split.predict(X_test_selected))

108645065486.96725

In [0]:
# All i could do for the knight.